<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A1 radar + cmls -> rain gauges 1 min  [xrl_yg1]
    


In [ ]:
# ! git clone https://github.com/nanopiero/fusion.git

In [1]:
import torch
import logging
logger = logging.getLogger("torch")
logger.setLevel(logging.ERROR)  # Show only errors, ignore warnings

import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# config de base (change en B.):
num_epochs = 2000
save_every = 10
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_A4_xrl_yg1.pt'
npoints = 10
npairs = 20
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

In [4]:
# Dataset, DataLoader
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=8)

In [5]:
ch_in = 72
ch_out = nsteps * 3
size = nsteps * 3

model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [6]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix
class QPELoss_fcn_train_A4(nn.Module):

    def __init__(self, eval_qpe_1h = False, eval_independent_qpe_1h=False, eval_segments=False):
        super(QPELoss_fcn_train_A4, self).__init__()
        self.regression_loss = nn.MSELoss()
        self.segmentation_loss = nn.CrossEntropyLoss()
        self.eval_qpe_1h = eval_qpe_1h
        self.eval_independent_qpe_1h = eval_independent_qpe_1h
        self.eval_segments = eval_segments

    def forward(self, p, outputs, targets, sampled_values_and_filters=None):

      # sans supervision imparfaite, avec spatialisation
      bs, nsteps, S, _ = targets.shape
      targets = targets.view(bs, 1, nsteps, S**2)
      mask = (targets>=0)
      masked_targets = targets[mask]
      outputs_rnr0 = outputs[:, :nsteps, ...].view(bs, 1, nsteps, S**2)
      outputs_rnr1 = outputs[:, nsteps:2*nsteps, ...].view(bs, 1, nsteps, S**2)
      masked_outputs_rnr = torch.cat([outputs_rnr0[mask].view(bs, 1, -1), outputs_rnr1[mask].view(bs, 1, -1)], dim=1)
      masked_targets_rnr =  (masked_targets > 0).long().view(bs, -1)
      outputs_qpe = outputs[:, 2*nsteps:3*nsteps , ...]\
                                .view(bs, 1, nsteps, S**2)

      masked_outputs_qpe = outputs_qpe[mask]
      masked_targets_qpe = masked_targets

      loss_rnr = self.segmentation_loss(masked_outputs_rnr, masked_targets_rnr)
      loss_qpe_1min = self.regression_loss(masked_outputs_qpe, masked_targets_qpe)
      loss = 1/(2*p[0]**2) * loss_qpe_1min + 1/(2*p[1]**2) * loss_rnr
      

      if self.eval_qpe_1h:
        loss_qpe_1h = self.regression_loss(outputs_qpe.sum(dim=2)[mask[:, :, 0, ...]],
                                           targets.sum(dim=2)[mask[:, :, 0, ...]])
        loss += 1/(2*p[2]**2) * loss_qpe_1h

      elif self.eval_independent_qpe_1h:
        outputs_qpe_1h = outputs[:, 3*nsteps, ...]\
                                .view(bs, 1, S**2)
          
        loss_qpe_1h = self.regression_loss(outputs_qpe_1h[mask[:, :, 0, ...]],
                                           targets.sum(dim=2)[mask[:, :, 0, ...]])   
        loss += 1/(2*p[2]**2) * loss_qpe_1h
      else:
        with torch.no_grad():
            loss_qpe_1h = self.regression_loss(outputs_qpe.sum(dim=2)[mask[:, :, 0, ...]],
                                               targets.sum(dim=2)[mask[:, :, 0, ...]])
          
          
      if self.eval_segments:
          sampled_values, filters = sampled_values_and_filters
          #Pas très joli : on aurait aimé appler outputs_QPE, mais il a été applati
          filtered_outputs = outputs[:, 2*nsteps:3*nsteps , ...].unsqueeze(dim=2) * \
                             filters.unsqueeze(dim=1)
          sampled_pred_values = torch.sum(filtered_outputs,\
                            dim=(3,4))
          loss_segments = self.regression_loss(sampled_pred_values, sampled_values)
          loss += 1/(2*p[3]**2) * loss_segments
      else:
          loss_segments = 0.

      loss += torch.log(1 + p[0]**2 + p[1]**2 + self.eval_qpe_1h * p[2]**2 + self.eval_independent_qpe_1h * p[2]**2 + self.eval_segments * p[3]**2)


      with torch.no_grad():
        preds = masked_outputs_rnr.argmax(dim=1).flatten().cpu().numpy()
        targets = masked_targets_rnr.flatten().cpu().numpy()
        # Compute the confusion matrix
        cm = confusion_matrix(targets, preds, labels=np.arange(2))

      if self.eval_segments:
          return loss_qpe_1min.item(), loss_qpe_1h.item(), loss_rnr.item(), loss, cm, loss_segments.item()
      else:
          return loss_qpe_1min.item(), loss_qpe_1h.item(), loss_rnr.item(), loss, cm, loss_segments



In [7]:
criterion_train = QPELoss_fcn_train_A4()
criterion_val = QPELoss_fcn()
# Baseline with a FCN
use_fcn = True

best_loss = [float('inf'), float('inf')]  # Initialize best validation loss to a very high value
# Boucle avec 5 modes d'évaluation
val_steps = ['val'
             ]
steps = val_steps + ['train']
losses = {step:[] for step in steps}
last_epoch = 0

In [8]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)
del checkpoint, model_weights, optimizer_state_dict

In [9]:
last_epoch

470

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model.train()
for epoch in range(last_epoch, num_epochs + 1):
  print('epoch n°', epoch)
  t = time.time()
  running_regression_loss = {step:0.0 for step in steps}
  running_regression_loss_1h = {step:0.0 for step in steps}
  running_segmentation_loss = {step:0.0 for step in steps}
  running_confusion_matrix = {step: np.zeros((2, 2), dtype=int) for step in steps}
    
  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)

    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)

    # for transformers :
    # segment_measurements = segment_gt(images, pairs, filters)
    _, segment_measurements_fcn = segment_gt(images, pairs, filters,
                                             use_fcn=use_fcn)

    # pseudo pluvios
    _, point_measurements_fcn, _ = point_gt(images, npoints=npoints,
                                            use_fcn=use_fcn)

    # pseudo radar
    noisy_images = make_noisy_images(images)

    # prepare inputs and targets
    inputs = torch.cat([noisy_images, segment_measurements_fcn], dim=1)
    targets = point_measurements_fcn

    # phase val
    step = 'val'
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion_val(model.p, outputs, targets)
        
    del outputs, loss
    torch.cuda.empty_cache()
      
    running_regression_loss[step] += regression_loss
    running_regression_loss_1h[step] += regression_loss_1h
    running_segmentation_loss[step] += segmentation_loss
    running_confusion_matrix[step] += batch_cm
      
    # phase train
    step = 'train'
    model.train()
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)  # Forward pass

    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion_train(model.p, outputs, targets)
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

      
    del inputs, targets, outputs, loss, noisy_images, images, pairs, filters
    torch.cuda.empty_cache()
    
    running_regression_loss[step] += regression_loss
    running_regression_loss_1h[step] += regression_loss_1h
    running_segmentation_loss[step] += segmentation_loss
    running_confusion_matrix[step] += batch_cm


  if epoch > 0:
    for step in steps:
      regression_loss = running_regression_loss[step] / len(loader)
      regression_loss_1h = running_regression_loss_1h[step] / len(loader)
      segmentation_loss = running_segmentation_loss[step] / len(loader)
      losses[step].append((epoch, regression_loss, regression_loss_1h, segmentation_loss, running_confusion_matrix[step]))

      print(f'{step}, Regression Loss: {regression_loss:.4f}, Regression Loss 1h: {regression_loss_1h:.4f}, Segmentation Loss:{segmentation_loss:.4f}' )
      print("Train Confusion Matrix:")
      print(running_confusion_matrix[step])
      accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(running_confusion_matrix[step])
      print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
      print('\n')
  print('epoch duration :', time.time() - t)


  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  

epoch n° 470
val, Regression Loss: 0.0502, Regression Loss 1h: 2.5552, Segmentation Loss:0.0855
Train Confusion Matrix:
[[3147906   54014]
 [  74873  563207]]
Accuracy: 0.9664, CSI: 0.8138, Sensitivity: 0.8827, Specificity: 0.9831, False Alarm Ratio: 0.0875


train, Regression Loss: 0.0115, Regression Loss 1h: 2.3122, Segmentation Loss:0.0813
Train Confusion Matrix:
[[3152068   49852]
 [  73336  564744]]
Accuracy: 0.9679, CSI: 0.8209, Sensitivity: 0.8851, Specificity: 0.9844, False Alarm Ratio: 0.0811


epoch duration : 73.49105072021484
saving step
epoch n° 471
val, Regression Loss: 0.0512, Regression Loss 1h: 2.3643, Segmentation Loss:0.0817
Train Confusion Matrix:
[[3151060   50851]
 [  72716  565373]]
Accuracy: 0.9678, CSI: 0.8206, Sensitivity: 0.8860, Specificity: 0.9841, False Alarm Ratio: 0.0825


train, Regression Loss: 0.0116, Regression Loss 1h: 2.2801, Segmentation Loss:0.0785
Train Confusion Matrix:
[[3152767   49144]
 [  69893  568196]]
Accuracy: 0.9690, CSI: 0.8268, Sensi